In [80]:
#create a virtual environment
!python3 -m venv venv

#install packages with pip to virtual environment
!./venv/bin/pip install groq langchain langchain-community sentence-transformers chromadb pypdf python-dotenv ipykernel google-genai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [81]:
#add virtual environment as a kernel to jupyter
!./venv/bin/python -m ipykernel install --user --name=ai_agent_env --display-name "AI Agent (Venv)"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Installed kernelspec ai_agent_env in /home/yyg/.local/share/jupyter/kernels/ai_agent_env


In [82]:
import os
import re
import json
import time
from datetime import datetime
from typing import List, Dict, Any
from dataclasses import dataclass, asdict
from dotenv import load_dotenv

#RAG imports
from groq import Groq
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

#gemini import
from google import genai
from google.genai import types


load_dotenv()

True

In [83]:
client = Groq(api_key=os.getenv("GROQ_API_KEY"))
gemini_client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

print("Clients initialized")


Clients initialized


In [84]:
print("Fetching PDF...")
loader = PyPDFLoader("kasko_kitapcigi_01_2024.pdf") 
pages = loader.load()
print(f"PDF loaded successfully - {len(pages)} pages")

#split into chunks with better parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", " ", ""]
)
chunks = text_splitter.split_documents(pages)
print(f"PDF chunked - {len(chunks)} chunks created")

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

vector_db = Chroma.from_documents(chunks, embeddings)
print("Vector database created successfully")

Fetching PDF...
PDF loaded successfully - 70 pages
PDF chunked - 685 chunks created
Vector database created successfully


In [85]:
def insurance_lookup_tool(query):
    print(f"\nSearching: '{query}'")
    
    #retrieve more results for better coverage
    docs = vector_db.similarity_search(query, k=7)
    
    if not docs:
        return "No relevant information found in document."
    
    #combine relevant chunks
    content = ""
    for i, doc in enumerate(docs):
        page_num = doc.metadata.get('page', 'N/A')
        content += f"\n[Source {i+1} - Page {page_num}]\n{doc.page_content}\n"
    
    #debug preview
    preview = content[:400].replace('\n', ' ')
    print(f"Found {len(docs)} results. Preview: {preview}...")
    
    return content

In [86]:
SYSTEM_PROMPT = """You are an expert AI assistant specializing in Anadolu Insurance (kasko) documents.
Your task is to answer user questions by using the 'insurance_lookup_tool' to search the document.

WORKFLOW (ReAct Pattern):
1. Thought: Analyze the question and determine which Turkish keywords to search for
2. Action: insurance_lookup_tool("short keywords") - Write query inside parentheses
3. Observation: Receive document search results
4. Repeat Thought/Action: Search with different terms if needed
5. Final Answer: Provide professional answer based ONLY on document information

CRITICAL RULES:
1. Use SHORT and CONCISE keywords (e.g., "fuel filling", "engine damage")
2. Prefer Turkish terms since document is in Turkish
3. NEVER fabricate limits/rates not in document
4. If info not found, clearly state so - don't guess
5. Action format: insurance_lookup_tool("search term")

EXAMPLE:
Thought: User is asking about wrong fuel filling damage. I'll search "wrong fuel" and "engine damage".
Action: insurance_lookup_tool("yanlış yakıt dolumu motor")
"""



In [87]:
def parse_action(response_text):
    patterns = [
        r'Action:\s*insurance_lookup_tool\(["\'](.+?)["\']\)',  # Action: insurance_lookup_tool("query")
        r'Action:\s*insurance_lookup_tool\((.+?)\)',             # Action: insurance_lookup_tool(query)
        r'Action:\s*["\'](.+?)["\']',                            # Action: "query"
        r'Action:\s*(.+?)(?:\n|$)',                              # Action: query
    ]
    
    for pattern in patterns:
        match = re.search(pattern, response_text, re.IGNORECASE)
        if match:
            query = match.group(1).strip().strip('"\'')
            return query
    
    return None


In [88]:
def run_insurance_agent(
    user_question,
    model="groq",
    max_steps=6,
    verbose=True
):
    
    if model == "gemini":
        messages = [
            {"role": "user", "content": SYSTEM_PROMPT + "\n\n" + user_question}
        ]
    else:
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_question}
        ]

    
    if verbose:
        print(f"\n{'='*60}")
        print(f"USER QUESTION: {user_question}")
        print(f"{'='*60}")
    
    for step in range(max_steps):
        #get response from model
        if model == "groq":
            completion = client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=messages,
                temperature=0.1,
                max_tokens=1000,
                stop=["Observation:", "Observation :"]
            )
            agent_response = completion.choices[0].message.content.strip()
            
        elif model == "gemini":
            contents = []
            for msg in messages:
                role = "user" if msg["role"] == "user" else "model"
                contents.append(types.Content(role=role, parts=[types.Part(text=msg["content"])]))

            response = gemini_client.models.generate_content(
                model="gemini-1.5-pro",
                contents=contents,
                config={
                    'temperature': 0.1,
                    'max_output_tokens': 1000,
                    'stop_sequences': ["Observation:", "Observation :"]
                }
            )
            agent_response = response.text.strip()

        else:
            raise ValueError(f"Unknown model: {model}")

        
        if verbose:
            print(f"\n{'─'*60}")
            print(f"STEP {step + 1}:")
            print(f"{'─'*60}")
            print(agent_response)
        
        #add agent response to memory
        messages.append({"role": "assistant", "content": agent_response})
        
        #check for Final Answer
        if "Final Answer:" in agent_response:
            final_answer = agent_response.split("Final Answer:")[-1].strip()
            if verbose:
                print(f"\n{'='*60}")
                print(f"RESULT:")
                print(f"{'='*60}")
                print(final_answer)
            return final_answer
        
        #check for Action
        if "Action:" in agent_response:
            query = parse_action(agent_response)
            
            if query:
                try:
                    #perform search
                    observation = insurance_lookup_tool(query)
                    
                    #add observation to system (truncated to prevent context overflow)
                    obs_message = f"Observation: Document search completed.\n{observation[:2000]}"
                    messages.append({"role": "assistant", "content": obs_message})
                    
                except Exception as e:
                    error_msg = f"Observation: Error during search: {str(e)}"
                    messages.append({"role": "assistant", "content": error_msg})
                    print(f" Error: {e}")
            else:
                #could not parse Action
                messages.append({
                    "role": "user", 
                    "content": "Please write Action in correct format: insurance_lookup_tool(\"search term\")"
                })
        else:
            #no Final Answer or Action, nudge the model
            messages.append({
                "role": "user",
                "content": "Please continue with Thought/Action loop or provide Final Answer."
            })
    
    return f"Couldn't reach definitive answer in {max_steps} steps. Please make question more specific."

In [89]:
#test question
question = "Aracıma yanlış yakıt dolumu yapıldı, motor yandı. Hasarım ödenir mi? Limitim nedir?"
print(run_insurance_agent(question))


USER QUESTION: Aracıma yanlış yakıt dolumu yapıldı, motor yandı. Hasarım ödenir mi? Limitim nedir?


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01kdntyrm5fd7vw3dwktknqrjd` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99871, Requested 293. Please try again in 2m21.696s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
@dataclass
class BenchmarkQuestion:
    id: int
    question: str
    ground_truth: str
    category: str  # "kapsam", "limit", "prosedür", "istisna"
    difficulty: str  # "easy", "medium", "hard"
    keywords: List[str]

@dataclass
class EvaluationResult:
    question_id: int
    model_name: str
    question: str
    response: str
    ground_truth: str
    correctness_score: float
    hallucination_detected: bool
    retrieval_quality: float
    completeness_score: float
    response_time: float
    keywords_found: List[str]
    keywords_missing: List[str]
    evaluation_notes: str

In [ ]:
class BenchmarkDataset:
    
    def __init__(self):
        self.questions = self._create_questions()
    
    def _create_questions(self) -> List[BenchmarkQuestion]:
        q_list = []
        
        # --- 1. EASY(1-15) ---
        q_list.extend([
            BenchmarkQuestion(1, "Yanlış yakıt dolumu sonucu motor hasarı ödenir mi?", "Evet, belirli bir limite kadar (poliçede yazar) teminat altındadır.", "kapsam", "easy", ["yanlış yakıt", "motor", "hasar"]),
            BenchmarkQuestion(2, "Cam kırılması kasko kapsamında mıdır?", "Evet, cam hasarları teminat altındadır.", "kapsam", "easy", ["cam", "kırılma"]),
            BenchmarkQuestion(3, "Mini onarım hizmeti neleri kapsar?", "Küçük ölçekli kaporta, boya, döşeme ve cam hasarlarını kapsar.", "hizmet", "easy", ["mini onarım", "kapsam"]),
            BenchmarkQuestion(4, "Aracım çalınırsa kasko ödeme yapar mı?", "Evet, hırsızlık ana teminattır.", "kapsam", "easy", ["çalınma", "hırsızlık"]),
            BenchmarkQuestion(5, "Yedek anahtar kaybı kasko kapsamında mı?", "Evet, kilit sisteminin değiştirilmesi teminata dahildir.", "kapsam", "easy", ["anahtar", "kayıp"]),
            BenchmarkQuestion(6, "İkame araç hizmeti kaç gündür?", "Poliçe türüne göre (genelde 7-15 gün) değişir.", "limit", "easy", ["ikame araç", "gün"]),
            BenchmarkQuestion(7, "Lastik patlaması kasko kapsamında mıdır?", "Sadece patlama değil, kaza ile birlikte hasar oluşursa ödenir.", "istisna", "easy", ["lastik", "patlama"]),
            BenchmarkQuestion(8, "Ferdi kaza teminatı vefat durumunu kapsar mı?", "Evet, poliçedeki limitler dahilinde kapsar.", "kapsam", "easy", ["ferdi kaza", "vefat"]),
            BenchmarkQuestion(9, "Deprem hasarları kaskoya dahil mi?", "Evet, kasko genel şartlarına ek olarak teminata dahildir.", "kapsam", "easy", ["deprem", "doğal afet"]),
            BenchmarkQuestion(10, "Sel hasarı sonucu araç pert olursa ne olur?", "Rayiç değer üzerinden ödeme yapılır.", "kapsam", "easy", ["sel", "pert"]),
            BenchmarkQuestion(11, "Yangın teminatı sadece kaza anındakileri mi kapsar?", "Hayır, park halindeki yanmaları da kapsar.", "kapsam", "easy", ["yangın", "yanma"]),
            BenchmarkQuestion(12, "Kemirgen hayvanların verdiği zararlar ödenir mi?", "Evet, fare vb. hayvanların kablo kemirmesi teminat altındadır.", "kapsam", "easy", ["kemirgen", "hayvan"]),
            BenchmarkQuestion(13, "Kasko poliçesi satış durumunda ne olur?", "Poliçe kendiliğinden sonlanır (münfesih olur).", "prosedür", "easy", ["satış", "devir"]),
            BenchmarkQuestion(14, "Hukuksal koruma teminatı limiti nedir?", "Poliçede belirtilen avukatlık ve mahkeme gideri limitidir.", "limit", "easy", ["hukuksal koruma", "limit"]),
            BenchmarkQuestion(15, "Orijinal cam değişimi her zaman mümkün müdür?", "Poliçe türüne (Muafiyetli/Muafiyetsiz) göre değişir.", "prosedür", "easy", ["orijinal cam", "servis"])
        ])

        # --- 2. MEDIUM (16-35) ---
        q_list.extend([
            BenchmarkQuestion(16, "Anahtar ile çalınma (evden çalınma) durumu kapsamda mı?", "Kilitli olmayan yerden veya zorlama olmadan çalınma şartlarına bağlıdır.", "kapsam", "medium", ["anahtar", "evden çalınma"]),
            BenchmarkQuestion(17, "Yurt dışında kaza yaparsam kaskom geçerli olur mu?", "Sadece Türkiye sınırları içinde geçerlidir, ek teminat gerekir.", "istisna", "medium", ["yurt dışı", "sınır"]),
            BenchmarkQuestion(18, "Aracın içindeki kişisel eşyaların çalınması ödenir mi?", "Belirli bir limite kadar ve kilitli araç şartıyla ödenir.", "limit", "medium", ["kişisel eşya", "çalınma"]),
            BenchmarkQuestion(19, "Hasar bildirim süresi kaç iş günüdür?", "Genel şartlara göre 5 iş günüdür.", "prosedür", "medium", ["bildirim", "süre", "ihbar"]),
            BenchmarkQuestion(20, "Özel servis ile yetkili servis farkı hasar ödemesini etkiler mi?", "Poliçede belirtilen servis ağı dışında kesinti yapılabilir.", "prosedür", "medium", ["özel servis", "yetkili servis"]),
            BenchmarkQuestion(21, "Eşdeğer parça kullanımı ne zaman başlar?", "Araç yaş ve parça türüne göre yasal mevzuata göre uygulanır.", "prosedür", "medium", ["eşdeğer parça", "yan sanayi"]),
            BenchmarkQuestion(22, "LPG takılan aracın hasarı ödenir mi?", "Ruhsata işli olması ve sigortacıya bildirilmiş olması gerekir.", "istisna", "medium", ["lpg", "tüp"]),
            BenchmarkQuestion(23, "Kusurlu olduğum kazada karşı tarafın hasarı nasıl ödenir?", "İMM (İhtiyari Mali Mesuliyet) limitleri dahilinde ödenir.", "limit", "medium", ["imm", "karşı taraf"]),
            BenchmarkQuestion(24, "Hatalı park yüzünden araç çekilirken hasar görürse ne olur?", "Çekici Anadolu Asistans ise teminat altındadır.", "kapsam", "medium", ["hatalı park", "çekici"]),
            BenchmarkQuestion(25, "Valeye teslim edilen aracın çalınması ödenir mi?", "İşletmenin sorumluluğundadır, kasko genellikle rücu eder veya reddeder.", "istisna", "medium", ["vale", "teslim"]),
            BenchmarkQuestion(26, "Terör olayları hasarları hangi şartla ödenir?", "Ek teminat olarak poliçeye eklenmiş olmalıdır.", "kapsam", "medium", ["terör", "eylem"]),
            BenchmarkQuestion(27, "Arıza nedeniyle yolda kalan araca hangi hizmet verilir?", "Çekici ve asistans hizmetleri verilir ancak parça ödenmez.", "hizmet", "medium", ["arıza", "yolda kalma"]),
            BenchmarkQuestion(28, "Hasarsızlık indirimi cam hasarında bozulur mu?", "Yılda bir kez cam hasarı indirimi bozmaz (poliçeye göre).", "prosedür", "medium", ["hasarsızlık", "bozulma"]),
            BenchmarkQuestion(29, "Araçta sigara yanığı oluşması kasko kapsamında mı?", "Genellikle ek teminatla veya 'dahili hasar' kapsamında değerlendirilir.", "kapsam", "medium", ["sigara yanığı", "döşeme"]),
            BenchmarkQuestion(30, "Kıymet kazanma tenzili nedir?", "Eski parçanın yerine yenisi takıldığında oluşan değer artış düşüşüdür.", "prosedür", "medium", ["kıymet kazanma", "eskime"]),
            BenchmarkQuestion(31, "Aracın pert olması için hasar oranı ne olmalıdır?", "Genellikle %50-%70 arası hasar ve onarım maliyeti bakılır.", "limit", "medium", ["pert", "hasar oranı"]),
            BenchmarkQuestion(32, "Ehliyetsiz araç kullanımı hasar ödemesini nasıl etkiler?", "Kesinlikle tazminat ödenmez, kasko dışıdır.", "istisna", "medium", ["ehliyetsiz", "red"]),
            BenchmarkQuestion(33, "Poliçe vadesi bitmeden iptal edilirse prim iadesi olur mu?", "Gün esasına göre iade yapılır.", "prosedür", "medium", ["iptal", "prim iadesi"]),
            BenchmarkQuestion(34, "Radyo-teyp çalınması teminata dahil midir?", "Orijinal ise dahildir, sonradan takıldıysa bildirilmelidir.", "kapsam", "medium", ["radyo", "teyp"]),
            BenchmarkQuestion(35, "Kıvılcım sıçraması sonucu oluşan hasarlar yangın sayılır mı?", "Evet, yangın teminatı kapsamında değerlendirilir.", "kapsam", "medium", ["kıvılcım", "yangın"])
        ])

        # --- 3. HARD (36-40) ---
        q_list.extend([
            BenchmarkQuestion(36, "Sel sırasında motoru çalıştırmak hasarın reddine yol açar mı?", "Brüt kusur ve hasarı ağırlaştırma gerekçesiyle red sebebi olabilir.", "istisna", "hard", ["sel", "motor çalıştırma", "su çekme"]),
            BenchmarkQuestion(37, "6 ay önce yapılan onarımın tekrarlaması durumunda garanti var mı?", "Anlaşmalı servislerde onarım garantisi poliçede belirtilir.", "istisna", "hard", ["tekrar hasar", "garanti"]),
            BenchmarkQuestion(38, "Alkollü araç kullanımında rücu şartları nelerdir?", "Zorunlu trafik sigortası ve kasko genel şartlarına göre belirlenir.", "istisna", "hard", ["alkol", "rücu", "promil"]),
            BenchmarkQuestion(39, "İstihap haddi (aşırı yük) aşımı hasarı nasıl etkiler?", "Hasar ile illiyet bağı varsa tazminat ödenmeyebilir.", "istisna", "hard", ["istihap haddi", "aşırı yük"]),
            BenchmarkQuestion(40, "Aracın ikinci el değer kaybı kasko tarafından karşılanır mı?", "Kasko sadece hasarı onarır, değer kaybı genellikle kapsam dışıdır.", "istisna", "hard", ["değer kaybı", "ikinci el"])
        ])

        # --- 4. OUT-OF-SCOPE (41-50) ---
        q_list.extend([
            BenchmarkQuestion(41, "2025 yılı zorunlu trafik sigortası tavan fiyatları nedir?", "Dokümanda bu bilgi bulunmamaktadır (Sadece Kasko içerir).", "out_of_scope", "hard", ["trafik sigortası", "fiyat"]),
            BenchmarkQuestion(42, "BMW F30 motor yağı kaç kilometrede değişir?", "Dokümanda teknik bakım kılavuzu bulunmamaktadır.", "out_of_scope", "hard", ["motor yağı", "bakım"]),
            BenchmarkQuestion(43, "Anadolu Sigorta hisseleri bugün borsada kaç TL?", "Güncel finansal veriler dokümanda yoktur.", "out_of_scope", "hard", ["borsa", "hisse"]),
            BenchmarkQuestion(44, "Konut sigortası kapsamında su tesisatı arızası ödenir mi?", "Bu kasko dokümanıdır, konut sigortasını kapsamaz.", "out_of_scope", "hard", ["konut sigortası", "tesisat"]),
            BenchmarkQuestion(45, "En iyi kasko şirketi hangisidir?", "Doküman objektif poliçe şartlarını içerir, kıyaslama yapmaz.", "out_of_scope", "hard", ["en iyi", "kıyas"]),
            BenchmarkQuestion(46, "Apple CarPlay bağlantı hatası nasıl düzeltilir?", "Teknik yazılım desteği kasko kapsamında değildir.", "out_of_scope", "hard", ["carplay", "bağlantı"]),
            BenchmarkQuestion(47, "Almanya plakalı araca Türkiye'den kasko yapılır mı?", "Doküman tescil mevzuatı içermez.", "out_of_scope", "hard", ["almanya", "plaka"]),
            BenchmarkQuestion(48, "Sigorta Genel Müdürü kimdir?", "Personel bilgisi dokümanda yer almamaktadır.", "out_of_scope", "hard", ["genel müdür", "kimdir"]),
            BenchmarkQuestion(49, "MTV ödemesi kredi kartına taksit yapılır mı?", "Vergi ödeme detayları sigorta poliçesinde yer almaz.", "out_of_scope", "hard", ["mtv", "taksit"]),
            BenchmarkQuestion(50, "Kaza sonrası karşı tarafa açılacak dava için avukat önerir misin?", "Doküman avukat tavsiyesi içermez.", "out_of_scope", "hard", ["avukat önerisi", "dava"])
        ])
        
        return q_list
    
    def get_by_category(self, category: str) -> List[BenchmarkQuestion]:
        return [q for q in self.questions if q.category == category]
    
    def get_by_difficulty(self, difficulty: str) -> List[BenchmarkQuestion]:
        return [q for q in self.questions if q.difficulty == difficulty]

In [ ]:
class ModelEvaluator:
    def evaluate_response(self, question: BenchmarkQuestion, response: str, 
                         response_time: float) -> EvaluationResult:
        
        keywords_found = [kw for kw in question.keywords 
                         if kw.lower() in response.lower()]
        keywords_missing = [kw for kw in question.keywords 
                           if kw.lower() not in response.lower()]
        
        completeness = len(keywords_found) / len(question.keywords) if question.keywords else 0
        
        hallucination_indicators = [
            "approximately", "around", "roughly", "I think", "probably",
            "tahminimce", "yaklaşık", "sanırım", "olabilir"
        ]
        hallucination_detected = any(ind in response.lower() for ind in hallucination_indicators)
        
        correctness_score = 0.7
        
        has_numbers = any(char.isdigit() for char in response)
        has_specific_terms = any(term in response.lower() for term in 
                                ["tl", "₺", "limit", "madde", "bent"])
        vague_terms = ["genel olarak", "genellikle", "çoğunlukla", "bazı durumlarda"]
        has_vague = any(term in response.lower() for term in vague_terms)
        
        retrieval_quality = 0.5
        if has_numbers: retrieval_quality += 0.2
        if has_specific_terms: retrieval_quality += 0.2
        if has_vague: retrieval_quality -= 0.3
        retrieval_quality = max(0.0, min(1.0, retrieval_quality))
        
        return EvaluationResult(
            question_id=question.id,
            model_name="",
            question=question.question,
            response=response,
            ground_truth=question.ground_truth,
            correctness_score=correctness_score,
            hallucination_detected=hallucination_detected,
            retrieval_quality=retrieval_quality,
            completeness_score=completeness,
            response_time=response_time,
            keywords_found=keywords_found,
            keywords_missing=keywords_missing,
            evaluation_notes=""
        )


In [ ]:
class BenchmarkRunner:
    
    def __init__(self, dataset: BenchmarkDataset, evaluator: ModelEvaluator):
        self.dataset = dataset
        self.evaluator = evaluator
        self.results: List[EvaluationResult] = []
    
    def run_benchmark(self, model_name: str, model_type: str, limit: int = None) -> List[EvaluationResult]:
        
        questions = self.dataset.questions[:limit] if limit else self.dataset.questions
        results = []
        
        print(f"\n{'='*70}")
        print(f"Running Benchmark for: {model_name}")
        print(f"Total Questions: {len(questions)}")
        print(f"{'='*70}\n")
        
        for i, question in enumerate(questions, 1):
            print(f"[{i}/{len(questions)}] Processing Q{question.id}...")
            
            start_time = time.time()
            
            try:
                response = run_insurance_agent(
                    question.question, 
                    verbose=False, 
                    model=model_type
                )
                response_time = time.time() - start_time
                
                eval_result = self.evaluator.evaluate_response(
                    question, response, response_time
                )
                eval_result.model_name = model_name
                
                results.append(eval_result)
                
                print(f"Completed in {response_time:.2f}s | Score: {eval_result.correctness_score:.2f}")
                
            except Exception as e:
                print(f"Error: {e}")
                continue
            
            time.sleep(0.5)
        
        self.results.extend(results)
        return results
    
    def generate_report(self, results: List[EvaluationResult]) -> Dict[str, Any]:
        
        if not results:
            return {
                "model": "UNKNOWN",
                "total_questions": 0,
                "metrics": {
                    "avg_correctness": 0.0,
                    "avg_completeness": 0.0,
                    "avg_retrieval_quality": 0.0,
                    "avg_response_time": 0.0,
                    "hallucination_rate": 0.0
                },
                "by_difficulty": {},
                "detailed_results": []
            }
        
        model_name = results[0].model_name
        
        avg_correctness = sum(r.correctness_score for r in results) / len(results)
        avg_completeness = sum(r.completeness_score for r in results) / len(results)
        avg_retrieval = sum(r.retrieval_quality for r in results) / len(results)
        avg_time = sum(r.response_time for r in results) / len(results)
        hallucination_rate = sum(1 for r in results if r.hallucination_detected) / len(results)
        
        easy_results = [r for r in results if any(
            q.id == r.question_id and q.difficulty == "easy" 
            for q in self.dataset.questions
        )]
        medium_results = [r for r in results if any(
            q.id == r.question_id and q.difficulty == "medium" 
            for q in self.dataset.questions
        )]
        hard_results = [r for r in results if any(
            q.id == r.question_id and q.difficulty == "hard" 
            for q in self.dataset.questions
        )]
        
        report = {
            "model": model_name,
            "timestamp": datetime.now().isoformat(),
            "total_questions": len(results),
            "metrics": {
                "avg_correctness": round(avg_correctness, 3),
                "avg_completeness": round(avg_completeness, 3),
                "avg_retrieval_quality": round(avg_retrieval, 3),
                "avg_response_time": round(avg_time, 2),
                "hallucination_rate": round(hallucination_rate, 3)
            },
            "by_difficulty": {
                "easy": {
                    "count": len(easy_results),
                    "avg_score": round(sum(r.correctness_score for r in easy_results) / len(easy_results), 3) if easy_results else 0
                },
                "medium": {
                    "count": len(medium_results),
                    "avg_score": round(sum(r.correctness_score for r in medium_results) / len(medium_results), 3) if medium_results else 0
                },
                "hard": {
                    "count": len(hard_results),
                    "avg_score": round(sum(r.correctness_score for r in hard_results) / len(hard_results), 3) if hard_results else 0
                }
            },
            "detailed_results": [asdict(r) for r in results]
        }
        
        return report
    
    def compare_models(self, reports: List[Dict[str, Any]]) -> Dict[str, Any]:
        
        comparison = {
            "timestamp": datetime.now().isoformat(),
            "models": [r["model"] for r in reports],
            "comparison": {}
        }
        
        metrics = ["avg_correctness", "avg_completeness", "avg_retrieval_quality", 
                  "avg_response_time", "hallucination_rate"]
        
        for metric in metrics:
            comparison["comparison"][metric] = {
                r["model"]: r["metrics"][metric] 
                for r in reports
            }
            
            if metric == "avg_response_time" or metric == "hallucination_rate":
                best_model = min(reports, key=lambda x: x["metrics"][metric])["model"]
            else:
                best_model = max(reports, key=lambda x: x["metrics"][metric])["model"]
            
            comparison["comparison"][metric]["best"] = best_model
        
        if len(reports) == 2:
            base_report = reports[0]
            comp_report = reports[1]
            
            comparison["percentage_differences"] = {}
            for metric in metrics:
                base_val = base_report["metrics"][metric]
                comp_val = comp_report["metrics"][metric]
                
                if base_val != 0:
                    diff = ((comp_val - base_val) / base_val) * 100
                    comparison["percentage_differences"][metric] = {
                        "difference": round(diff, 2),
                        "interpretation": f"{comp_report['model']} is {abs(diff):.1f}% {'better' if diff > 0 else 'worse'}"
                    }
        
        return comparison
    
    def save_results(self, filepath: str):
        
        models = list(set(r.model_name for r in self.results))
        reports = []
        
        for model in models:
            model_results = [r for r in self.results if r.model_name == model]
            report = self.generate_report(model_results)
            reports.append(report)
        
        comparison = self.compare_models(reports) if len(reports) > 1 else None
        
        output = {
            "reports": reports,
            "comparison": comparison
        }
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(output, f, indent=2, ensure_ascii=False)
        
        print(f"\nResults saved to: {filepath}")
    
    def print_summary(self, report: Dict[str, Any]):
        if report.get('model') == "UNKNOWN" or 'metrics' not in report:
            print("\n!!! HATA: Model hiçbir soruyu yanıtlayamadı, rapor oluşturulamadı. !!!")
            return
        
        
        print(f"\n{'='*70}")
        print(f"BENCHMARK RESULTS: {report['model']}")
        print(f"{'='*70}")
        print(f"Total Questions: {report['total_questions']}")
        print(f"\nMETRICS:")
        print(f"  Correctness:        {report['metrics']['avg_correctness']:.1%}")
        print(f"  Completeness:       {report['metrics']['avg_completeness']:.1%}")
        print(f"  Retrieval Quality:  {report['metrics']['avg_retrieval_quality']:.1%}")
        print(f"  Avg Response Time:  {report['metrics']['avg_response_time']:.2f}s")
        print(f"  Hallucination Rate: {report['metrics']['hallucination_rate']:.1%}")
        
        print(f"\nBY DIFFICULTY:")
        for diff in ["easy", "medium", "hard"]:
            data = report['by_difficulty'][diff]
            if data['count'] > 0:
                print(f"  {diff.capitalize():8} ({data['count']:2} questions): {data['avg_score']:.1%}")


In [ ]:
test_questions = [
    "Aracıma yanlış yakıt dolumu yapıldı, motor yandı. Hasarım ödenir mi? Limitim nedir?",
    "Kasko sigortam cam kırılmalarını karşılar mı?",
    "Mini onarım hizmeti nedir?",
]

for i, question in enumerate(test_questions):
    print(f"\n\n{'#'*60}")
    print(f"TEST {i+1}")
    print(f"{'#'*60}")
    result = run_insurance_agent(question, verbose=True)

In [ ]:
print("### Manuel Sorgu Testi ###")
test_results = vector_db.similarity_search("yakıt", k=3)
for i, res in enumerate(test_results):
    print(f"Result {i+1} (Page {res.metadata.get('page')}):")
    print(res.page_content)
    print("-" * 30)

In [ ]:
#initialize benchmark system
dataset = BenchmarkDataset()
evaluator = ModelEvaluator()
runner = BenchmarkRunner(dataset, evaluator)

#test with Groq Llama 3.3
groq_results = runner.run_benchmark(
    model_name="Groq Llama-3.3-70B",
    model_type="groq",
    limit=5
)
groq_report = runner.generate_report(groq_results)
runner.print_summary(groq_report)


In [ ]:
#test with Gemini Pro (first 5 questions)
gemini_results = runner.run_benchmark(
    model_name="Google Gemini-1.5-Pro",
    model_type="gemini",
    limit=5
)
gemini_report = runner.generate_report(gemini_results)
runner.print_summary(gemini_report)

In [ ]:
#generate comparison report
comparison = runner.compare_models([groq_report, gemini_report])

print("\n" + "="*70)
print("MODEL COMPARISON")
print("n" + "="*70)
print(json.dumps(comparison, indent=2, ensure_ascii=False))


In [ ]:
#save all results to JSON file
runner.save_results("benchmark_results.json")
print("\nBenchmark completed! Check benchmark_results.json for details.")

In [ ]:
user_input = input("Sigorta ile ilgili sorunuzu yazın: ")
print("\n" + "="*60)
answer = run_insurance_agent(user_input, verbose=True)